![](pngs/nba_html_to_csv.png)
1. Define Foundation
2. NBA Seasons To CSV

### ➤ 1 Define Foundation 

In [1]:
import os
import sys
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
DATA_PATH = r"C:\Users\knaue\Documents\Data\NBA"
SEASON_HTML_PATH = os.path.join(DATA_PATH, "SEASON_HTML")

SEASON_DETAIL_PATH = os.path.join(DATA_PATH, "Season_Detail_Urls.csv")
NBA_SEASON_DETAIL_PATH = os.path.join(DATA_PATH, "NBA_Season_Detail.csv")

PARSER = 'lxml'

In [3]:
# Some infomations are hidden in comments --> convert comments in normal html code
def filter_out_comment(soup: BeautifulSoup) -> BeautifulSoup:
    content = str(soup).replace('<!--', '')
    content = content.replace('-->', '')
    return BeautifulSoup(content, PARSER)

In [4]:
def get_season_summary(df: pd.DataFrame, url: str):
    html_path = os.path.join(SEASON_HTML_PATH, url.replace("/", "{").replace(":", "}"))
    dfs = []
    
    with open(html_path, 'r', encoding="utf-8-sig") as f:
        content = BeautifulSoup(f.read(), PARSER)
        
        table_team = content.find("table", id="per_game-team")
        table_team_opp = content.find("table", id="per_game-opponent")
        table_team_adv = content.find("table", id="advanced-team")
        
        for idx, table in enumerate([table_team, table_team_opp, table_team_adv]): 
            df_team = pd.read_html(str(table))[0]
            if idx == 2: 
                df_team = df_team.droplevel(0, axis=1)
            
            df_team = df_team.drop([col for col in df_team.columns if "Unnamed:" in col], axis="columns", inplace=False)
            del df_team['Rk']
            
            df_team = df_team[:-1]
            df_team['Team'] = df_team['Team'].str.replace("\*", "", regex=True)
            
            if idx == 1: 
                df_team.columns = [f"{col}_opp" if col != "Team" else col for col in df_team.columns]
            
            dfs.append(df_team)   
            
        f.close()
        
    for df_table in dfs: 
        df = df.merge(right=df_table, how="left", on="Team", validate="one_to_one")   
    return df   

In [5]:
def get_season_standing(df: pd.DataFrame, url: str):
    html_path = os.path.join(SEASON_HTML_PATH, url.replace("/", "{").replace(":", "}"))
    dfs = []
    
    with open(html_path, 'r', encoding="utf-8-sig") as f:
        content = filter_out_comment(BeautifulSoup(f.read(), PARSER))
        table_team = content.find("table", id="expanded_standings")
        df_team = pd.read_html(str(table_team))[0]
        
        df_team = df_team.droplevel(0, axis=1)
        df_team = df_team.drop([col for col in df_team.columns if col not in ['Team', 'Overall', 'Home', 'Road', 'Pre', 'Post', '≤3', '≥10', 'Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr']], 
                               axis="columns", 
                               inplace=False)
        
        df = df.merge(right=df_team, how="left", on="Team", validate="one_to_one")
        f.close()
        
    return df

In [6]:
def champion_in_percent(cell: float):
    if cell == 0: return 0 
    if cell == 1: return 100
    if cell == 2: return 50
    if cell == 3 or cell == 4: return 25
    if cell >= 5 and cell <= 8: return 12.5
    if cell >= 9 and cell <= 16: return 6.25
    if cell >= 17 and cell <= 32: return 3.125

In [7]:
def get_playoff_standing(df: pd.DataFrame, url: str):
    html_path = os.path.join(SEASON_HTML_PATH, url.replace("/", "{").replace(":", "}"))
    dfs = []
    
    with open(html_path, 'r', encoding="utf-8-sig") as f:
        content = filter_out_comment(BeautifulSoup(f.read(), PARSER))
        table_team = content.find("table", id="expanded_standings")
        df_team = pd.read_html(str(table_team))[0]
        
        df_team = df_team.droplevel(0, axis=1)
        df_team = df_team.drop([col for col in df_team.columns if col not in ['Rk', 'Team']], 
                               axis="columns", 
                               inplace=False)
        
        df = df.merge(right=df_team, how="left", on="Team", validate="one_to_one")
        df['Rk'] = df['Rk'].fillna(0)
        df['Champion_Percent'] = df['Rk'].apply(lambda x: champion_in_percent(cell=x))
        f.close()
           
    return df

### ➤ 2 NBA Seasons To CSV

In [8]:
df_season_detail = pd.read_csv(SEASON_DETAIL_PATH, encoding="utf-8-sig")
df_season_detail = df_season_detail.drop([col for col in df_season_detail.columns if "Unnamed:" in col], axis="columns", inplace=False)
df_season_detail = df_season_detail[df_season_detail['Lg'] == "NBA"]

df_season_sum = []
df_season_sta = []
df_playoff_sta = []

# Season Summary-------------------------------------------------------------------------------
print("Season Summary")
i = 0
for url_season_sum in df_season_detail['Url_Season_Summary'].unique(): 
    i += 1
    sys.stdout.write(f"\r{i}/{len(df_season_detail['Url_Season_Summary'].unique())}...")
    
    df = df_season_detail[df_season_detail['Url_Season_Summary'] == url_season_sum]
    df = get_season_summary(df[['Team', 'Season', 'Lg']], url_season_sum)
    df_season_sum.append(df)

# Season Standings-----------------------------------------------------------------------------   
print("\nSeason Standings")
i = 0     
for url_season_sta in df_season_detail['Url_Season_Standings'].unique():
    i += 1
    sys.stdout.write(f"\r{i}/{len(df_season_detail['Url_Season_Standings'].unique())}...")
    
    df = df_season_detail[df_season_detail['Url_Season_Standings'] == url_season_sta]
    df = get_season_standing(df[['Team', 'Season', 'Lg']], url_season_sta)
    df_season_sta.append(df)

# Playoff Standings----------------------------------------------------------------------------    
print("\nPlayoff Standings")
i = 0     
for url_playoff_sta in df_season_detail['Url_Playoff_Standings'].unique():
    i += 1
    sys.stdout.write(f"\r{i}/{len(df_season_detail['Url_Playoff_Standings'].unique())}...")
    
    df = df_season_detail[df_season_detail['Url_Playoff_Standings'] == url_playoff_sta]
    df = get_playoff_standing(df[['Team', 'Season', 'Lg']], url_playoff_sta)
    df_playoff_sta.append(df)


#Merge Summary and Standings-------------------------------------------------------------------
df_season_sum = pd.concat(df_season_sum, ignore_index=True)
df_season_detail = df_season_detail.merge(right=df_season_sum, how="left", on=['Team', 'Season', 'Lg'], validate="one_to_one")

df_season_sta = pd.concat(df_season_sta, ignore_index=True)
df_season_detail = df_season_detail.merge(right=df_season_sta, how="left", on=['Team', 'Season', 'Lg'], validate="one_to_one")

df_playoff_sta = pd.concat(df_playoff_sta, ignore_index=True)
df_season_detail = df_season_detail.merge(right=df_playoff_sta, how="left", on=['Team', 'Season', 'Lg'], validate="one_to_one")


df_season_detail.to_csv(NBA_SEASON_DETAIL_PATH, index=False, encoding="utf-8-sig")
print("\nSaved to: ", NBA_SEASON_DETAIL_PATH)

Season Summary
73/73...
Season Standings
73/73...
Playoff Standings
73/73...
Saved to:  C:\Users\knaue\Documents\Data\NBA\NBA_Season_Detail.csv
